In [5]:
import os
from os.path import join
import numpy as np
from cell_mrcnn.utils import get_weights_path_from_config_file, \
    get_data_path_from_config_file
from matplotlib import pyplot as plt
from imgaug import augmenters as iaa
import tensorflow as tf

# Import Mask RCNN
from cell_mrcnn.config import Config
from cell_mrcnn import model as modellib, utils


# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = join(get_weights_path_from_config_file(), "logs")
dataset_dir = join(get_data_path_from_config_file(), 'annotated_datasets')

############################################################
#  Configurations
############################################################


class CellConfig(Config):
    """Configuration for training on the 2 channel confocal dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "cell"

    IMAGE_CHANNEL_COUNT = 2
    MEAN_PIXEL = np.array([4.5, 1.5])

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + cell

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 200
    # todo: 0.98 might be too high
    DETECTION_MIN_CONFIDENCE = 0.98

    # Length of square anchor side in pixels
    # Each scale is associated with a level of the pyramid
    #todo: determine the appropriate anchor scales (basically u need to
    # understand how the feaeture pyramide network and how the
    # utils/generate_pyramid_anchors() work)
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    # RPN proposals w/ an IoU  higher, than threshold get deleted, so lower
    # threshold => more filtered.
    # we don't want to detect overlapping cells (so you'd lower this)
    RPN_NMS_THRESHOLD = 0.4

    # to reduce memory load
    BACKBONE = "resnet18"
    DIV_FILTERNB = 2
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 562
    POST_NMS_ROIS_INFERENCE = 1024
    RPN_TRAIN_ANCHORS_PER_IMAGE = 128

    # Random crops of size
    IMAGE_RESIZE_MODE = "crop"
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024

    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    # MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask

    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    TRAIN_ROIS_PER_IMAGE = 128

    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 100

    # Max number of final detections per image
    DETECTION_MAX_INSTANCES = 100

    # Number of validation steps to run at the end of every training epoch.
    # A bigger number improves accuracy of validation stats, but slows
    # down the training.
    VALIDATION_STEPS = 50


class CellInferenceConfig(CellConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # Don't resize images for inferencing
    IMAGE_RESIZE_MODE = "pad64"


############################################################
#  Dataset
############################################################


class CellDataset(utils.Dataset):

    def load_cell(self, path_to_dataset, subset):
        """Load the cell images.
        path_to_dataset: Path to the dataset that contaions the annotated
        images.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("cell", 1, "cell")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        path_to_dataset = os.path.join(path_to_dataset, subset)

        subfolders = [os.path.join(path_to_dataset, subfolder) for subfolder in
                      os.listdir(path_to_dataset)]

        for subfolder in subfolders:
            image_names = [file_name for file_name in os.listdir(subfolder) if
                           'mask' not in file_name]
            # Add images
            for image_name in image_names:
                id = image_name.split('.')[0]
                im_path = subfolder + '/' + image_name
                self.add_image(
                    "cell",
                    image_id=id, # use file name as a unique image id
                    path=im_path)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a cell dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "cell":
            return super(self.__class__, self).load_mask(image_id)

        im_path = self.image_info[image_id]['path']
        subfolder = '/'.join(im_path.split('/')[:-1]) + '/'
        mask_names = [file_name for file_name in os.listdir(subfolder)
                      if 'mask' in file_name]
        mask = None
        for mask_name in mask_names:
            mask_path = subfolder + mask_name
            mask_ = plt.imread(mask_path)
            # add new dim so shape is (512,512,1) the 3rd dim is for different
            # objects
            mask_ = mask_[..., np.newaxis]
            if mask is None:
                mask = mask_
            else:
                mask = np.concatenate((mask, mask_), axis=2)


        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "cell":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


def train(model):
    """Train the model."""
    dataset_path = join(dataset_dir, '2020_11_22_02_55_03/')
    # Training dataset.
    dataset_train = CellDataset()
    dataset_train.load_cell(dataset_path, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CellDataset()
    dataset_val.load_cell(dataset_path, "val")
    dataset_val.prepare()

    # Image augmentation
    # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    augmentation = iaa.SomeOf((0, 2), [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.OneOf([iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
        iaa.GaussianBlur(sigma=(0.0, 1.0)),
        iaa.Add((-10, 10)),
        iaa.Multiply((0.8, 1.5))
    ])

    print("Train all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=35,
                augmentation=augmentation,
                layers='all')

In [6]:
config = CellConfig()
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=DEFAULT_LOGS_DIR)

# load the last model trained
weights_path = model.find_last()
model.load_weights(weights_path, by_name=True)

In [7]:
train(model)

Train all layers

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/miska/repos/cell_mrcnn/logs/cell20201018T1936/mask_rcnn_cell_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2b          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
res3a_branch1          (Conv2D)
bn3a_branch2b          (BatchNorm)
bn3a_branch1           (BatchNorm)
res3b_branch2a         (Conv2D)
bn3b_branch2a          (BatchNorm)
res3b_branch2b         (Conv2D)
bn3b_branch2b          (BatchNorm)
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b        

/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'







Epoch 1/50
200/200 [==============================] - 209s 1s/step - loss: 2.4144 - rpn_class_loss: 0.1841 - rpn_bbox_loss: 0.8606 - mrcnn_class_loss: 0.2329 - mrcnn_bbox_loss: 0.5749 - mrcnn_mask_loss: 0.5619 - val_loss: 2.0466 - val_rpn_class_loss: 0.0883 - val_rpn_bbox_loss: 0.6297 - val_mrcnn_class_loss: 0.2187 - val_mrcnn_bbox_loss: 0.5213 - val_mrcnn_mask_loss: 0.5886

Epoch 2/50
 54/200 [=======>......................] - ETA: 1:06 - loss: 2.2198 - rpn_class_loss: 0.1207 - rpn_bbox_loss: 0.6022 - mrcnn_class_loss: 0.3173 - mrcnn_bbox_loss: 0.5604 - mrcnn_mask_loss: 0.6191

Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/miska/anaconda3/envs/cell_mrcnn/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/miska/anaconda3/envs/cell_mrcn

KeyboardInterrupt: 